# Shapley effects for the Rust Model

In [1]:
# Import statements as in simulation_convergence.ipynb.
import matplotlib.pyplot as plt
#import yaml
import numpy as np
from ruspy.simulation.simulation import simulate
from ruspy.model_code.fix_point_alg import calc_fixp
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.estimation.estimation_transitions import create_transition_matrix

# My imports.
from ruspy.estimation.estimation import estimate
from ruspy.model_code.demand_function import get_demand 
from python.econsa_shapley import get_shapley

## Understand simulation capabilities of `ruspy`
The below code is taken from the notebook simulation_convergence provided as promotional material for `ruspy`.

In [15]:
# Set simulating variables
disc_fac = 0.9999
num_buses = 50
num_periods = 120
gridsize = 1000
# We use the cost parameters and transition probabilities from the replication
params = np.array([10.07780762, 2.29417622])
trans_probs = np.array([0.39189182, 0.59529371, 0.01281447])
scale = 1e-3

init_dict_sim = {
    "simulation": {
        "discount_factor": disc_fac,
        "periods": num_periods,
        "seed": 123,
        "buses": num_buses,
    },
    "plot": {"gridsize": gridsize},
}

In [17]:
# Calucalte objects necessary for the simulation process. See documentation for details.
num_states = 200
costs = calc_obs_costs(num_states, lin_cost, params, scale)

trans_mat = create_transition_matrix(num_states, trans_probs)
ev = calc_fixp(trans_mat, costs, disc_fac)[0]

In [18]:
%%time
# Simulate the data.
df = simulate(init_dict_sim["simulation"], ev, costs, trans_mat)

Wall time: 24 ms


In [19]:
df.head()

state  decision  utilities  usage
Bus_ID period                                   
1      0           0         0  -0.753076      1
       1           1         0  -0.819370      0
       2           1         0  -0.723300      1
       3           2         0  -0.848635      1
       4           3         0  -0.876315      1

Bring data in form required for `ruspy`'s estimate function: drop column 'utilities'. Then estimate structural parameters, i.e. $\hat{\theta}$. 

In [20]:
data = df[['state', 'decision', 'usage']].copy()

In [21]:
data.head()

state  decision  usage
Bus_ID period                        
1      0           0         0      1
       1           1         0      0
       2           1         0      1
       3           2         0      1
       4           3         0      1

In [22]:
# Can use one init_dict for get_demand() and estimate().
init_dict_estimation = {
    'model_specifications': {
        'discount_factor': disc_fac,
        'number_states': num_states,
        'maint_cost_func': 'linear',
        'cost_scale': 1e-3
    },
    'optimizer': {
        'approach': 'NFXP',
        'algorithm': 'scipy_L-BFGS-B',
        'gradient': 'Yes'
    },
    
}

In [23]:
%%time
result_transitions_nfxp, result_fixp_nfxp = estimate(init_dict_estimation, data)

Wall time: 2.72 s


In [24]:
result_transitions_nfxp

{'trans_count': array([2372, 3556,   72], dtype=int64),
 'x': array([0.39533333, 0.59266667, 0.012     ]),
 'fun': 4379.948672063236}

In [25]:
result_fixp_nfxp

{'x': array([10.97720291,  2.57833076]),
 'fun': 203.80690327991155,
 'status': 1,
 'message': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'jac': array([ 1.22209003e-05, -2.78977068e-05]),
 'n_evaluations': 13,
 'n_iterations': 12,
 'n_contraction_steps': 280,
 'n_newt_kant_steps': 128,
 'time': 2.6111743999999817}

In [13]:
estimate_structural_parameters = np.concatenate((result_transitions_nfxp['x'], result_fixp_nfxp['x']))

In [14]:
estimate_structural_parameters

array([ 0.39156025,  0.59561925,  0.0128205 , 10.11119702,  2.31405273])